In [ ]:
import numpy as np
import os
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

import netCDF4
import time
import xarray

In [ ]:
""" Read .nc file into pandas dataframe."""
#Runtime around 452 seconds using perf_counter.

tic = time.perf_counter()

nc_ibtracs = 'Allstorms.ibtracs_all.v03r10.nc'
path_to_nc = os.path.join('..','deconstruct_cyn',nc_ibtracs)

ibt_all_fields = xarray.open_dataset(path_to_nc).to_dataframe()
# name, storm_sn, time_record, landfall, 
# genesis_basin, season, 
# lat, lon, max_wind, min_pres, dist2land are the interesting variables

columns_we_want = ['storm_sn',
                   'name',
                   'source_time',
                   'landfall',
                   'genesis_basin',
                   'season',
                   'source_lat',
                   'source_lon',
                   'source_wind',
                   'source_pres',
                   'dist2land']

ibt = ibt_all_fields[columns_we_want]

del ibt_all_fields

renamed_columns = ['storm_sn',
                   'name',
                   'time_recorded',
                   'landfall',
                   'genesis_basin',
                   'season',
                   'lat',
                   'lon',
                   'max_wind',
                   'min_pres',
                   'dist2land']

columns = dict(zip(columns_we_want, renamed_columns))

ibt2 = ibt.rename(index=str, columns = columns)

del ibt

toc = time.perf_counter()

elapsed = toc - tic
print(elapsed)
#ibt

In [ ]:
""" Import landmask."""
#Elapsed time ~10 seconds.
tic = time.process_time()

nc_landmask = 'ETOPO1_Ice_g_gmt4.nc' #Land mask from ETOPO1 Global Relief Model
path_to_nc2 = os.path.join('..','deconstruct_cyn',nc_landmask)

landmask_all_fields = xarray.open_dataset(path_to_nc2).to_dataframe()

toc = time.process_time()

elapsed = toc - tic
print(elapsed)

# Creating a Boolean Mask for the Phillipines

Here we want to create a bounding box for the phillipines. 

Following the coordinates of the previous groups box, so we can replicate their results.

(117.4E, 18.1N),
(122.9E,19.2N),
(128.2E, 6.2N),
(122.6E, 3.5N).

Note that the box made by the previous project team had sloped sides, so this approach will work for an arbitrary quadrilateral.

#### Algorithm

Split the quadrilateral into three parts, a(lpha), b(eta), gamma. 
The quadrilateral has four corners.

Alpha:
Select points with lattitudes between the highest lattitude corner to second highest lattitude corner.
Select points with longitudes as a function of lattitude between these two points in the quadrilateral.

Beta:
Select points with lattitudes between the 2nd highest lattitude corner to 3rd highest lattitude corner.
Select points with longitudes as a function of lattitude between these two points in the quadrilateral.

Gamma:
Select points with lattitudes between the 3rd highest lattitude corner to lowest lattitude corner.
Select points with longitudes as a function of lattitude between these two points in the quadrilateral.

Take the union of the three sets to get set of points in box.

In [ ]:
lm2 = landmask_all_fields.reset_index()

In [ ]:
m1 = 1/5     # top line gradient. 
m2 = -130/53 # right line gradient.
m3 = 27/56   # bottom line gradient.
m4 = -73/26  # left line gradient.

c1 = -5.38   # top line "y-intercept"
c2 = +320.65 # etc.
c3 = -55.61
c4 = +347.72

In [ ]:
lat_within_a = (18.1 <= lm2.lat) & (lm2.lat <= 19.2)
lat_within_a.value_counts()

In [ ]:
lon_left_a = (lm2.lat - c1)/m1
lon_right_a = (lm2.lat - c2)/m2
lon_within_a = (lon_left_a <= lm2.lon) & (lm2.lon <= lon_right_a)
lon_within_a.value_counts()

In [ ]:
point_within_a = lat_within_a & lon_within_a
point_within_a.value_counts()

In [ ]:
lat_within_b = (6.2 <= lm2.lat) & (lm2.lat <= 18.1)
lat_within_b.value_counts()

In [ ]:
lon_left_b = (lm2.lat - c4)/m4
lon_right_b = (lm2.lat - c2)/m2
lon_within_b = (lon_left_b <= lm2.lon) & (lm2.lon <= lon_right_b)
lon_within_b.value_counts()

In [ ]:
point_within_b = lat_within_b & lon_within_b
point_within_b.value_counts()

In [ ]:
lat_within_gamma = (3.5 <= lm2.lat) & (lm2.lat <= 6.2)
lat_within_gamma.value_counts()

In [ ]:
lon_left_gamma = (lm2.lat - c4)/m4
lon_right_gamma = (lm2.lat - c3)/m3
lon_within_gamma = (lon_left_gamma <= lm2.lon) & (lm2.lon <= lon_right_gamma)
lon_within_gamma.value_counts()

In [ ]:
point_within_gamma = lat_within_gamma & lon_within_gamma
point_within_gamma.value_counts()



In [ ]:
point_within_phillipines_box = (point_within_a | point_within_b | point_within_gamma)
point_within_phillipines_box.value_counts()

In [ ]:
lm2['phil_box'] = point_within_phillipines_box

# Selecting Correct Cyclone Events

So we have cyclone centres with lat, lon values. We want to measure whether a cyclone has reached land. A simple approach would be to check when the eye of the storm comes into contact with land.

Here I am trying to merge all the etopo data into the ibtracs database before I filter cyclone measurements within the phillipines bounding box. This leads to 14,999 cyclone measurements being selected within the box, for all non-null z (z can be negative).

In [ ]:
ibt2.head()

In [ ]:
lm2.head()

In [ ]:
ibt2.reset_index(inplace = True) # I want centre, storm and time in my final .pkl file of cyclones in phillipines box.
ibt2.head()

In [ ]:
N = 10000

# ibt2['lat'] and ibt2['lon'] columns have a float datatype.
# It is very difficult to run a pd.merge() on these columns.
# The float has rounding errors, running np.allclose() won't
# fix these,

# To fix this, multiply by a big power of 10, convert to nullable integer type.
# pd.merge() on integer indexed columns.
# Then convert back to float.

ibt2['lat'] = np.round(ibt2['lat']* N).astype(pd.Int64Dtype())
ibt2['lon'] = np.round(ibt2['lon']* N).astype(pd.Int64Dtype())

lm2['lat'] = np.round(lm2['lat'] * N).astype(pd.Int64Dtype())
lm2['lon'] = np.round(lm2['lon']* N).astype(pd.Int64Dtype())

flag_merge = pd.merge(ibt2, lm2, how = 'left', on = ['lat', 'lon'])

ibt2['lat'] = ibt2['lat']/N;
ibt2['lon'] = ibt2['lon']/N;

lm2['lat'] = lm2['lat']/N;
lm2['lon'] = lm2['lon']/N;

flag_merge['lat'] = flag_merge['lat']/N;
flag_merge['lon'] = flag_merge['lon']/N;

In [ ]:
boxfall = flag_merge[(flag_merge.phil_box == True)]

In [ ]:
boxfall.to_pickle(os.path.join('..','deconstruct_cyn','phil_cyclones.pkl'))